In [1]:
import tensorflow as tf

from tensorflow.keras.datasets import mnist, cifar10, cifar100
from tensorflow.keras.datasets import fashion_mnist

from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Activation

from sklearn.model_selection import train_test_split

import numpy as np
import random

import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider

In [2]:
#load the data
(x, y), (x_test, y_test) = mnist.load_data()
#(x, y), (x_test, y_test) = fashion_mnist.load_data()

labels = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']
img_rows, img_cols, channels = 28, 28, 1
num_classes = 10

#process the data
x, x_test = x / 255.0, x_test / 255.0

x = x.reshape((-1, img_rows, img_cols, channels))
x_test = x_test.reshape((-1, img_rows, img_cols, channels))

y = tf.keras.utils.to_categorical(y, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

#split training set, test set, attack set
x_train, x_attack, y_train, y_attack = train_test_split(x, y, test_size=0.1)

print("Data shapes:", x_test.shape, y_test.shape, x_train.shape, y_train.shape, x_attack.shape, y_attack.shape)

4423680/4422102 [==============================] - 0s 0us/step
Data shapes: (10000, 28, 28, 1) (10000, 10) (54000, 28, 28, 1) (54000, 10) (6000, 28, 28, 1) (6000, 10)


In [3]:
#training model
def create_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), strides=(3, 3), padding='same', activation='relu', input_shape=(img_rows, img_cols, channels)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), strides=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(200))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
    
    return model

victim_model = create_model()
victim_model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data = (x_test, y_test))
victim_model.save('model/victim_model')
#victim_model.save('model/victim_model_fashion')


Epoch 1/10
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0397 - accuracy: 0.6923 - val_loss: 0.0243 - val_accuracy: 0.8275
Epoch 2/10
1688/1688 [==============================] - 4s 2ms/step - loss: 0.0213 - accuracy: 0.8522 - val_loss: 0.0211 - val_accuracy: 0.8568
Epoch 3/10
1688/1688 [==============================] - 4s 2ms/step - loss: 0.0189 - accuracy: 0.8698 - val_loss: 0.0192 - val_accuracy: 0.8685
Epoch 4/10
1688/1688 [==============================] - 4s 2ms/step - loss: 0.0181 - accuracy: 0.8770 - val_loss: 0.0194 - val_accuracy: 0.8649
Epoch 5/10
1688/1688 [==============================] - 4s 2ms/step - loss: 0.0165 - accuracy: 0.8869 - val_loss: 0.0196 - val_accuracy: 0.8660
Epoch 6/10
1688/1688 [==============================] - 4s 2ms/step - loss: 0.0163 - accuracy: 0.8895 - val_loss: 0.0178 - val_accuracy: 0.8780
Epoch 7/10
1688/1688 [==============================] - 4s 2ms/step - loss: 0.0156 - accuracy: 0.8950 - val_loss: 0.0204 - val_accuracy:

In [4]:
#testing model
print("Base accuracy on regular images:", victim_model.evaluate(x=x_test, y=y_test, verbose=0))

Base accuracy on regular images: [0.01766335405409336, 0.8788999915122986]


In [5]:
#prediction API
def get_prediction(image):
    return victim_model.predict(image).argmax()

In [6]:
#Create attacker model architecture
def create_attack_model(model_arch):
    if model_arch == 5:
        model = Sequential()
        model.add(Conv2D(32, kernel_size=(3, 3), strides=(3, 3), padding='same', activation='relu', input_shape=(img_rows, img_cols, channels)))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Conv2D(64, kernel_size=(3, 3), strides=(3, 3), padding='same', activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Flatten())
        model.add(Dense(200))
        model.add(Dense(100))
        model.add(Dense(num_classes, activation='softmax'))

    if model_arch == 4:
        model = Sequential()
        model.add(Conv2D(32, kernel_size=(3, 3), strides=(3, 3), padding='same', activation='relu', input_shape=(img_rows, img_cols, channels)))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Conv2D(64, kernel_size=(3, 3), strides=(3, 3), padding='same', activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Flatten())
        model.add(Dense(200))
        model.add(Dense(num_classes, activation='softmax'))

    if model_arch == 3:
        model = Sequential()
        model.add(Conv2D(32, kernel_size=(3, 3), strides=(3, 3), padding='same', activation='relu', input_shape=(img_rows, img_cols, channels)))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Flatten())
        model.add(Dense(200))
        model.add(Dense(num_classes, activation='softmax'))

    if model_arch == 2:
        model = Sequential()
        model.add(Flatten())
        model.add(Dense(200))
        model.add(Dense(num_classes, activation='softmax'))

    if model_arch == 1:
        model = Sequential()
        model.add(Flatten())
        model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
    
    return model

In [7]:
#creating adversarial example
def adversarial_pattern(image, label, model):
    image = tf.cast(image, tf.float32)
    
    with tf.GradientTape() as tape:
        tape.watch(image)
        prediction = model(image)
        loss = tf.keras.losses.MSE(label, prediction)
    
    gradient = tape.gradient(loss, image)
    
    signed_grad = tf.sign(gradient)
    
    return signed_grad

In [8]:
#Training substitute model
def train_attack_model(iteration, init_num, model_arch=4, eps=64/255.0):

    print("Attack start!")
    counter = 1

    #Collect initial data
    x_attack_init_ind = random.sample(range(0, x_attack.shape[0]), init_num)
    x_attack_init = x_attack[x_attack_init_ind]

    #Label initial data
    y_attack_init = []
    for image in x_attack_init:
        y_attack_init.append(get_prediction(image.reshape(1, img_rows, img_cols, channels)))

    y_attack_init = tf.one_hot(y_attack_init, 10)

    #select attack model architecture
    attack_model = create_attack_model(model_arch)
    attack_model.fit(x_attack_init, y_attack_init, batch_size=32, epochs=10, validation_data = (x_test, y_test))
    print("Finish inital model training!")
    print("Attack model accuracy: %f" % (attack_model.evaluate(x_test, y_test)[1]))
    print("Training substitute model start!")

    x_attack_set = x_attack_init
    y_attack_set = y_attack_init
    
    #start training
    while True:
        print("Iteration: %d" % (counter))
        print("creating synthetic example...")

        for i in range(x_attack_set.shape[0]):
            image = x_attack_set[i]
            image_label = y_attack_set[i]
            perturbations = adversarial_pattern(image.reshape((1, img_rows, img_cols, channels)), image_label, attack_model).numpy()
            adversarial = image + perturbations * eps
            x_attack_set = np.append(x_attack_set, adversarial, axis = 0)

        #Label new dataset
        print("Labeling new dataset...")
        y_attack_set = []
        for image in x_attack_set:
            y_attack_set.append(get_prediction(image.reshape(1, img_rows, img_cols, channels)))

        y_attack_set = tf.one_hot(y_attack_set, 10)

        #train the attack model with new dataset
        attack_model.fit(x_attack_set, y_attack_set, batch_size=32, epochs=10, validation_data = (x_test, y_test))

        #evluate the attack model
        print("Number of query: %d" % (x_attack_set.shape[0]))
        print("Attack model accuracy: %f" % (attack_model.evaluate(x_test, y_test)[1]))

        if counter == iteration:
            print("Training finish!")
            if model_arch == 4:
                model_path = 'model/attack_model/i'+str(iteration)+'s'+str(init_num)
                #model_path = 'model/attack_model_fashion/i'+str(iteration)+'s'+str(init_num)
            else:
                model_path = 'model/attack_model/mismatch_layer'+str(model_arch)
                #model_path = 'model/attack_model_fashion/mismatch_layer'+str(model_arch)
            attack_model.save(model_path)
            break

        counter += 1

In [10]:
#start attack: train_attack_model(iteration, init_num, model_arch, eps)
train_attack_model(3, 200)

Attack start!
Epoch 1/10
7/7 [==============================] - 1s 76ms/step - loss: 0.0901 - accuracy: 0.1241 - val_loss: 0.0891 - val_accuracy: 0.2052
Epoch 2/10
7/7 [==============================] - 0s 55ms/step - loss: 0.0884 - accuracy: 0.2074 - val_loss: 0.0877 - val_accuracy: 0.2631
Epoch 3/10
7/7 [==============================] - 0s 53ms/step - loss: 0.0865 - accuracy: 0.2922 - val_loss: 0.0851 - val_accuracy: 0.3240
Epoch 4/10
7/7 [==============================] - 0s 54ms/step - loss: 0.0825 - accuracy: 0.3720 - val_loss: 0.0807 - val_accuracy: 0.3396
Epoch 5/10
7/7 [==============================] - 0s 54ms/step - loss: 0.0760 - accuracy: 0.3937 - val_loss: 0.0753 - val_accuracy: 0.4084
Epoch 6/10
7/7 [==============================] - 0s 56ms/step - loss: 0.0689 - accuracy: 0.4817 - val_loss: 0.0694 - val_accuracy: 0.4516
Epoch 7/10
7/7 [==============================] - 0s 53ms/step - loss: 0.0612 - accuracy: 0.5150 - val_loss: 0.0651 - val_accuracy: 0.5089
Epoch 8/10
7/